In [3]:
from keras import applications

# build the VGG16 network
model = applications.VGG16(include_top=False,
                           weights='imagenet')

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [8]:
from __future__ import print_function

import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block3_conv2'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())


kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

# we will stich the best 64 filters on a 8 x 8 grid.
n = 4

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
#for i in range(n):
 #   for j in range(n):
  #      img, loss = kept_filters[i * n + j]
   #     width_margin = (img_width + margin) * i
    #    height_margin = (img_height + margin) * j
     #   stitched_filters[
      #      width_margin: width_margin + img_width,
       #     height_margin: height_margin + img_height, :] = img

# save the result to disk
save_img('stitched_filters_%dx%d3.png' % (n, n), stitched_filters)


Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0      

Current loss value: 2970.3682
Current loss value: 3251.968
Current loss value: 3525.583
Current loss value: 3793.715
Current loss value: 4057.6768
Current loss value: 4318.0806
Current loss value: 4574.553
Current loss value: 4828.4644
Current loss value: 5080.7046
Current loss value: 5331.1646
Current loss value: 5580.367
Current loss value: 5828.6475
Filter 8 processed in 6s
Processing filter 9
Current loss value: 32.056477
Current loss value: 106.01211
Current loss value: 215.76428
Current loss value: 349.90747
Current loss value: 489.60474
Current loss value: 634.59454
Current loss value: 778.2478
Current loss value: 919.6485
Current loss value: 1057.6975
Current loss value: 1191.9746
Current loss value: 1323.4119
Current loss value: 1451.9165
Current loss value: 1578.2129
Current loss value: 1702.846
Current loss value: 1825.4001
Current loss value: 1946.467
Current loss value: 2066.2646
Current loss value: 2185.1636
Current loss value: 2303.8164
Current loss value: 2421.8237
Filt

Current loss value: 1312.5051
Current loss value: 1400.8933
Current loss value: 1457.1404
Current loss value: 1531.3473
Current loss value: 1586.0234
Current loss value: 1655.2612
Current loss value: 1709.5474
Current loss value: 1776.1624
Current loss value: 1831.1523
Current loss value: 1895.7991
Current loss value: 1950.5391
Current loss value: 2012.8896
Current loss value: 2067.2524
Current loss value: 2127.143
Current loss value: 2181.059
Current loss value: 2238.831
Filter 21 processed in 6s
Processing filter 22
Current loss value: 308.24377
Current loss value: 603.67993
Current loss value: 744.145
Current loss value: 819.5235
Current loss value: 921.9133
Current loss value: 991.6161
Current loss value: 1080.4985
Current loss value: 1149.4836
Current loss value: 1227.933
Current loss value: 1294.3971
Current loss value: 1366.3143
Current loss value: 1431.7102
Current loss value: 1500.3604
Current loss value: 1565.3452
Current loss value: 1632.4072
Current loss value: 1696.7079
Cu

Current loss value: 265.02728
Current loss value: 517.7977
Current loss value: 703.031
Current loss value: 857.0267
Current loss value: 1017.844
Current loss value: 1171.8584
Current loss value: 1322.3875
Current loss value: 1466.9373
Current loss value: 1605.8503
Current loss value: 1739.448
Current loss value: 1871.0126
Current loss value: 1997.9617
Current loss value: 2123.678
Current loss value: 2245.2915
Current loss value: 2367.6523
Current loss value: 2487.0537
Current loss value: 2605.1643
Current loss value: 2722.0122
Current loss value: 2838.0603
Current loss value: 2952.708
Filter 34 processed in 10s
Processing filter 35
Current loss value: 192.70459
Current loss value: 336.31192
Current loss value: 463.31775
Current loss value: 579.32434
Current loss value: 687.5228
Current loss value: 791.0729
Current loss value: 892.3699
Current loss value: 990.1154
Current loss value: 1085.0569
Current loss value: 1178.4956
Current loss value: 1269.8987
Current loss value: 1360.5703
Curr

Current loss value: 1426.6697
Current loss value: 1482.7024
Current loss value: 1537.2896
Filter 46 processed in 10s
Processing filter 47
Current loss value: 531.056
Current loss value: 719.47
Current loss value: 890.3956
Current loss value: 1051.0023
Current loss value: 1199.6147
Current loss value: 1340.4644
Current loss value: 1472.1649
Current loss value: 1596.667
Current loss value: 1713.3016
Current loss value: 1823.8829
Current loss value: 1929.0774
Current loss value: 2030.9269
Current loss value: 2130.0972
Current loss value: 2226.171
Current loss value: 2320.983
Current loss value: 2415.2554
Current loss value: 2508.7402
Current loss value: 2600.603
Current loss value: 2691.999
Current loss value: 2782.5593
Filter 47 processed in 10s
Processing filter 48
Current loss value: 213.82993
Current loss value: 387.07077
Current loss value: 532.0857
Current loss value: 659.7721
Current loss value: 776.7568
Current loss value: 886.2743
Current loss value: 990.5925
Current loss value: 

...

Current loss value: 1537.9934
Current loss value: 1615.7006
Current loss value: 1692.8135
Current loss value: 1771.5726
Filter 139 processed in 11s
Processing filter 140
Current loss value: 126.56584
Current loss value: 274.3317
Current loss value: 397.64767
Current loss value: 478.1973
Current loss value: 547.3767
Current loss value: 607.4386
Current loss value: 668.0023
Current loss value: 724.1753
Current loss value: 782.26227
Current loss value: 837.2705
Current loss value: 896.5714
Current loss value: 952.48267
Current loss value: 1013.47205
Current loss value: 1071.8274
Current loss value: 1134.6207
Current loss value: 1194.7429
Current loss value: 1258.6565
Current loss value: 1319.9329
Current loss value: 1384.1437
Current loss value: 1447.0487
Filter 140 processed in 11s
Processing filter 141
Current loss value: 282.9199
Current loss value: 486.8098
Current loss value: 628.2624
Current loss value: 747.60693
Current loss value: 857.7317
Current loss value: 964.8351
Current loss

Current loss value: 1907.0709
Current loss value: 2025.9421
Current loss value: 2142.8933
Current loss value: 2258.5396
Current loss value: 2373.0322
Current loss value: 2486.9531
Current loss value: 2601.186
Current loss value: 2713.9036
Current loss value: 2826.2903
Current loss value: 2937.7158
Filter 152 processed in 11s
Processing filter 153
Current loss value: 223.7458
Current loss value: 355.60452
Current loss value: 447.9278
Current loss value: 529.91895
Current loss value: 602.9205
Current loss value: 666.3174
Current loss value: 721.3839
Current loss value: 772.70074
Current loss value: 818.4695
Current loss value: 862.673
Current loss value: 905.79236
Current loss value: 945.8538
Current loss value: 986.7913
Current loss value: 1023.8833
Current loss value: 1064.2742
Current loss value: 1099.667
Current loss value: 1139.2643
Current loss value: 1173.3994
Current loss value: 1212.1985
Current loss value: 1245.9302
Filter 153 processed in 11s
Processing filter 154
Current loss

Current loss value: 864.6559
Current loss value: 966.6006
Current loss value: 1062.6509
Current loss value: 1155.6506
Current loss value: 1243.2732
Current loss value: 1329.1595
Current loss value: 1407.2474
Current loss value: 1485.4773
Current loss value: 1559.6182
Current loss value: 1629.1052
Current loss value: 1695.2324
Current loss value: 1758.7314
Current loss value: 1820.5103
Current loss value: 1879.5144
Current loss value: 1938.365
Current loss value: 1995.1318
Filter 165 processed in 11s
Processing filter 166
Current loss value: 465.8947
Current loss value: 1011.40375
Current loss value: 1439.041
Current loss value: 1826.5422
Current loss value: 2197.2773
Current loss value: 2555.1484
Current loss value: 2901.456
Current loss value: 3232.1074
Current loss value: 3549.6343
Current loss value: 3856.0374
Current loss value: 4153.7173
Current loss value: 4445.4795
Current loss value: 4730.797
Current loss value: 5010.6543
Current loss value: 5285.385
Current loss value: 5556.05

Current loss value: 443.5529
Current loss value: 717.458
Current loss value: 936.0536
Current loss value: 1123.0388
Current loss value: 1290.3766
Current loss value: 1442.5417
Current loss value: 1583.3608
Current loss value: 1718.0082
Current loss value: 1848.3217
Current loss value: 1973.2471
Current loss value: 2095.3594
Current loss value: 2214.8125
Current loss value: 2332.4854
Current loss value: 2447.5156
Current loss value: 2562.648
Current loss value: 2676.2354
Current loss value: 2788.5918
Current loss value: 2899.7515
Current loss value: 3010.6516
Current loss value: 3121.2947
Filter 178 processed in 11s
Processing filter 179
Current loss value: 206.18816
Current loss value: 408.37708
Current loss value: 559.33923
Current loss value: 680.64233
Current loss value: 777.8849
Current loss value: 859.9762
Current loss value: 932.93335
Current loss value: 1002.28156
Current loss value: 1067.4943
Current loss value: 1128.5422
Current loss value: 1186.4672
Current loss value: 1242.8

Current loss value: 1995.9302
Current loss value: 2091.9648
Current loss value: 2187.6548
Filter 190 processed in 12s
Processing filter 191
Current loss value: 75.85474
Current loss value: 139.6835
Current loss value: 219.42867
Current loss value: 301.01947
Current loss value: 412.986
Current loss value: 482.81604
Current loss value: 582.37054
Current loss value: 634.21045
Current loss value: 721.8109
Current loss value: 767.17786
Current loss value: 844.8866
Current loss value: 891.2539
Current loss value: 962.25586
Current loss value: 1005.57446
Current loss value: 1070.5743
Current loss value: 1114.894
Current loss value: 1175.5625
Current loss value: 1219.4503
Current loss value: 1275.9194
Current loss value: 1321.9946
Filter 191 processed in 11s
Processing filter 192
Current loss value: 65.90768
Current loss value: 120.956566
Current loss value: 164.65958
Current loss value: 202.20909
Current loss value: 241.28693
Current loss value: 279.70398
Current loss value: 320.24493
Current

NameError: name 'model' is not defined